<a href="https://colab.research.google.com/github/ivans14/Deep_Learning_Proj/blob/main/CNN_using_VGGM_for_ball_and_player_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ball and players detection for SoccerNet
### Using CNN and the pretained model VGGM

In [1]:
%matplotlib tk
import numpy as np
import tkinter
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import seaborn as sns
from sklearn import metrics
import torch.optim as optim
import torch
from torch import nn
from torch.nn import LocalResponseNorm
import pandas as pd
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
import cv2
import random
from files.engine import train_one_epoch, evaluate, hi

import files.utils as utils
import files.transforms as T
from tkinter import *
# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import metrics

In [9]:
hi()

hi


False

In [ ]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))

In [ ]:
# Import dataset as: train, validation and test splits

# data = pd.read_csv('proj_test/proj_det/det.txt')
# images= ['proj_test/proj_img1/{}.jpg'.format(str(i).zfill(6)) for i in range(1,750)]
# images[1]
# int(images[1][-10:-4])

# Data split
directory_train = 'proj_test/Train'

# Load Train data
train_labels = pd.read_csv(directory_train + "/proj_det/det.txt", sep=',')
train_images = directory_train + "/proj_img1/{}.jpg"
train_images = [train_images.format(str(i).zfill(6)) for i in range(1,751)] # train_labels.index.values
train_labels = [train_labels.columns.values.tolist()] + train_labels.values.tolist() # Convert DF to list

#print(train_labels[0:5][0:7])

# Load Test data
directory_test = 'proj_test/Test'

test_labels = pd.read_csv(directory_test + '/proj_det/det.txt', sep=',', index_col=0)
test_images = directory_test + "/proj_img1/{}.jpg"
test_images = [test_images.format(str(i).zfill(6)) for i in range(1,751)]
test_labels = [test_labels.columns.values.tolist()] + test_labels.values.tolist() # Convert DF to list

In [5]:
img = cv2.imread(train_images[1])
print(img.shape)

(1080, 1920, 3)


In [4]:
# Create image dataset class
class img_dataset(torch.utils.data.Dataset):
    def __init__(self, height, width, dir_img, labels_list, images, transforms=None, mode='train') -> None:
        self.transforms = transforms
        self.height = height
        self.width = width
        self.dir_img = dir_img
        self.labels_list = labels_list
        self.images=images

    def __getitem__(self,id):
        boxes = []
        labels = []
        img_name = self.images[id]
        print(img_name)

        # reading the images and converting them to correct size and color    
        img = cv2.imread(img_name)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        ## NOTE: If img.shape = (1080, 1920, 3), then no need to resize bboxes.
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        img_res /= 255.0
        
        for line in range(1, len(self.labels_list)):
            #parsed = [float(x) for x in line.split(',')]
            if int(self.labels_list[line][0]) == int(self.images[id][-10:-4]):
                x = self.labels_list[line][2]
                y = self.labels_list[line][3]
                width = self.labels_list[line][4]
                height = self.labels_list[line][5]
                x_max = x + width
                y_max = y + height
                boxes.append([x,y,x_max,y_max])
                if int(self.labels_list[line][-1]) == -1:
                    labels.append(1)
                elif int(self.labels_list[line][-1]) == 1:
                    labels.append(2)
                else:
                    labels.append(self.labels_list[line][-1])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        print(boxes)
        print(boxes.shape)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target["iscrowd"] = iscrowd
        target["area"] = area
        image_id = torch.tensor([id])
        target["image_id"] = image_id
        target["iscrowd"] = iscrowd

        if self.transforms:
            sample = self.transforms(image = img_res,
                                        bboxes = target['boxes'],
                                        labels = labels)
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            return img_res, target

        return img_res,boxes,target
         

    def __len__(self):
        return len(self.images)

In [5]:
# Create transformations fo the dataset to increase the data.
# Send train=True for training transforms and False for val/test transforms
def get_transform(train):
  if train:
    return A.Compose(
      [
        A.HorizontalFlip(0.5),
        # ToTensorV2 converts image to pytorch tensor without div by 255
        ToTensorV2(p=1.0) 
      ],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )
  else:
    return A.Compose(
      [ToTensorV2(p=1.0)],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )

In [6]:
train_ds = img_dataset(1080,1920,directory_train + "/proj_img1/",train_labels,train_images, transforms=get_transform(train=True))
test_ds = img_dataset(1080,1920,directory_test + "/proj_img1/",test_labels,test_images, mode='test', transforms=get_transform(train=False))
ran_img, boxes = train_ds[20]
ran_img.shape

proj_test/Train/proj_img1/000021.jpg
tensor([[ 929.,  489.,  974.,  626.],
        [ 402.,  344.,  435.,  450.],
        [   4.,  328.,   41.,  431.],
        [ 252.,  538.,  307.,  696.],
        [ 756.,  530.,  808.,  672.],
        [ 298.,  866.,  399., 1079.],
        [  25.,  666.,  159.,  836.],
        [ 114.,  333.,  148.,  426.],
        [ 736.,  244.,  770.,  321.],
        [1536.,  525., 1577.,  663.],
        [1254.,  610., 1276.,  634.],
        [1230.,  607., 1283.,  792.]])
torch.Size([12, 4])


torch.Size([3, 1080, 1920])

In [9]:
# Visualizig data TEST 02/12/2022
# Visualize a single image and its bounding boxes.

def show_random_image_boxes(img_dataset):
    idx_2print = random.randint(1, len(img_dataset.images))
    ran_img, boxes = img_dataset[idx_2print]

    img_2plot = ((ran_img.permute(1,2,0)).numpy()).copy()

    fig = plt.figure(figsize=(10,8))
    boxes_list = (boxes.get("boxes")).tolist()
    target_list = (boxes.get("labels")).tolist()   

    for i in range((boxes.get("boxes").shape)[0]):
        x = int(boxes_list[i][0])
        y = int(boxes_list[i][1])
        x_max = int(boxes_list[i][2])
        y_max = int(boxes_list[i][3])
        if (target_list[i] == 1):
            cv2.rectangle(img_2plot, (x,y),(x_max,y_max),(255,0,0),5)
            cv2.putText(img= img_2plot, text = str(target_list[i]), org = (x, y),fontFace = cv2.FONT_HERSHEY_TRIPLEX, fontScale = 2, color = (255,0,0), thickness= 2, lineType=cv2.LINE_AA)
        elif (target_list[i] == 2): # Rectangle for ball in blue
            cv2.rectangle(img_2plot, (x,y),(x_max,y_max),(0,0,255),6)
            cv2.putText(img= img_2plot, text = str(target_list[i]), org = (x, y),fontFace = cv2.FONT_HERSHEY_TRIPLEX, fontScale = 2, color = (0,0,255), thickness= 2, lineType=cv2.LINE_AA)
        
    plt.imshow(img_2plot)
    plt.axis('off')
    plt.title("Figure num. " + str(((boxes.get("image_id")).tolist())[0]))
    fig.show

In [12]:
show_random_image_boxes(train_ds)

proj_test/Train/proj_img1/000255.jpg
tensor([[1676.,  234., 1726.,  337.],
        [1398.,  624., 1497.,  826.],
        [1026.,   64., 1049.,  126.],
        [ 603.,  520.,  664.,  688.],
        [1024.,  247., 1065.,  354.],
        [1280.,  765., 1385., 1003.]])
torch.Size([6, 4])


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 

## DATALOADER

In [23]:
# dataset = img_dataset(1080,1920,'proj_test/proj_img1/',transforms=get_transform(train=True))
# dataset_test = img_dataset(1080,1920,'proj_test/proj_img1/',transforms=get_transform(train=True))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(train_ds)).tolist()

# train test split
test_split = 0.2
tsize = int(len(train_ds)*test_split)
dataset = torch.utils.data.Subset(train_ds, indices[:10])
dataset_test = torch.utils.data.Subset(train_ds, indices[-tsize:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
  train_ds,
  batch_size=1,
  shuffle=True,
  num_workers=0,
  collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
  test_ds,
  batch_size=10,
  shuffle=False,
  num_workers=0,
  collate_fn=utils.collate_fn
)

In [20]:
# Show a few train images TEST
import random
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


images, targets = next(iter(data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

for number in random.sample([1,2,3],3):
    boxes = targets[number]['boxes'].cpu().numpy().astype(np.int32)
    img = images[number].permute(1,2,0).cpu().numpy()
    labels = targets[number]['labels'].cpu().numpy().astype(np.int32)
    fig, ax = plt.subplots(1, 1, figsize=(16,8))

    for i in range (len(boxes)):
        img = cv2.rectangle(img, (boxes[i][0],boxes[i][1]), (boxes[i][2],boxes[i][3]),(255,0,0),2)
        img =cv2.putText(img, labels[i], (int(boxes[i][0]), int(boxes[i][1])),cv2.FONT_HERSHEY_TRIPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

    ax.set_axis_off()
    ax.imshow(img)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


PRETRAINED MODEL


In [13]:
## Faster RESNET 50 pre-trained model :)
def get_object_detection_model(num_classes):
  # load a model pre-trained pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
  return model

TRAINING


In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2 #CLASS ZERO FOR BACKGROUND, 1 FOR PLAYERS AND 2 FOR BALL 

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(
  optimizer,
  step_size=3,
  gamma=0.1
)

In [15]:
torch.cuda.is_available()

True

In [16]:
# training for 5 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

# Save model (i.e. its weights)
models_dir = "trained_models/"
torch.save(model.state_dict(), models_dir + 'CNN_weights_{}.pth'.format(len([entry for entry in os.listdir(models_dir) if os.path.isfile(os.path.join(models_dir,entry))])+1))
print("Finished training.")

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# Define training function
# HOLA 
# adios

def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

# Set number of epochs according to computational power, time and results.
batch_size = 64
num_epochs = 4
validation_every_steps = 500

step = 0
NN.train()

train_accuracies = []
valid_accuracies = []
running_loss = 0



for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    train_loss_batches = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass, compute gradients, perform one training step.
        # Your code here!
        output = NN(inputs) # Pass the inouts through the NN
        loss = loss_fn(output,targets) # Compute and Save loss
        optimizer.zero_grad() #Clean up gradients
        loss.backward() # Compute gradients based on the loss from the current batch
        optimizer.step()
        running_loss += loss.item()

        # Increment step counter
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy(targets, predictions))
        
        if step % validation_every_steps == 0:
            
            validation_loss = 0
            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))
            
            train_accuracies_batches = []
        
            # Compute accuracies on validation set.
            valid_accuracies_batches = []
            with torch.no_grad():
                NN.eval()
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    output = NN(inputs)
                    loss = loss_fn(output, targets)

                    predictions = output.max(1)[1]

                    # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    valid_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

                NN.train()
                
            # Append average validation accuracy to list.
            valid_accuracies.append(np.sum(valid_accuracies_batches) / len(test_set))
     
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
            print(f"             test accuracy: {valid_accuracies[-1]}")

# Save model (i.e. its weights)
models_dir = "trained_models/"
torch.save(model.state_dict(), models_dir + 'CNN_weights.pth')
print("Finished training.")